<a href="https://colab.research.google.com/github/simsekergun/DieRingSolver/blob/main/Tidy3D_Ring_ModeSolver_ppw_vs_neff.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tidy3d

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.2/85.2 kB 7.3 MB/s eta 0:00:00
  Attempting uninstall: h5netcdf
    Found existing installation: h5netcdf 1.6.3
    Uninstalling h5netcdf-1.6.3:
      Successfully uninstalled h5netcdf-1.6.3


In [2]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import BoundaryNorm
from matplotlib.ticker import MaxNLocator
import time
from numpy import random
import matplotlib.pyplot as plt
import matplotlib as mpl
import psutil
import os

# tidy3D import
import tidy3d.web as web
import tidy3d as td
from tidy3d.plugins.mode import ModeSolver
from tidy3d.plugins.mode.web import run as run_mode_solver

In [15]:
ppws = [10,15,20,25,30,35,40,80,100]
#ppws = [10]

for i in range(len(ppws)):
  start_time = time.perf_counter()
  # Start memory tracking
  process = psutil.Process(os.getpid())

  # Unit length is micron.
  wg_height = 0.67
  wg_width = 0.89
  # central radius of the simulated ring
  radius = 23.0-wg_width/2.0

  # Waveguide and substrate materials
  mat_wg = td.Medium(permittivity=2.0221636**2.0)
  mat_background = td.Medium(permittivity=1.449679**2.0)

  # Free-space wavelength (in um) and frequency (in Hz)
  lambda0 = 1.06
  freq0 = td.C_0 / lambda0
  fwidth = freq0 / 10

  # Simulation size inside the PML along propagation direction
  sim_length = 2.4*radius
  # Simulation domain size, resolution and total run time
  sim_size = [sim_length, 2.0*sim_length, 2.0*sim_length]
  run_time = 80 / fwidth
  grid_spec = td.GridSpec.auto(min_steps_per_wvl=ppws[i], wavelength=lambda0)


  # The ring is made by two cylinders
  cyl1 = td.Structure(
      geometry=td.Cylinder(
          center=[0, 0, 0],
          radius=radius - wg_width / 2,
          length=wg_height,
          axis=2,
      ),
      #medium=td.Medium(),
      medium=mat_background,
  )
  cyl2 = td.Structure(
      geometry=td.Cylinder(
          center=[0, 0, 0],
          radius=radius + wg_width / 2,
          length=wg_height,
          axis=2,
      ),
      medium=mat_wg,
  )

  # xy-plane frequency-domain field monitor; slightly offset in z for better structure viz below
  field_mnt = td.FieldMonitor(
      center=[0, 0, 0], size=[td.inf, td.inf, 0], freqs=[freq0], name="field"
  )

  # Flux monitor along the ring propagation direction
  flux_mnt = td.FluxMonitor(
      center=[0, radius, 0], size=[0, 5*wg_height, 5*wg_width], freqs=[freq0], name="flux"
  )

  # Simulation
  sim = td.Simulation(
      center=[1.1*radius, 0, 0],
      size=sim_size,
      medium=mat_background,
      grid_spec=grid_spec,
      structures=[cyl2, cyl1],
      sources=[],
      monitors=[field_mnt, flux_mnt],
      run_time=run_time,
      boundary_spec=td.BoundarySpec(x=td.Boundary.absorber(), y=td.Boundary.pml(), z=td.Boundary.pml())
  )

  # Modal source plane
  source_plane = td.Box(center=[0, -radius, 0], size=[0, 5*wg_height, 5*wg_width])

  num_modes = 20
  # NB: negative radius since the plane position is at y=-radius
  mode_spec = td.ModeSpec(num_modes=num_modes, bend_radius=-radius, bend_axis=1)
  ms = ModeSolver(simulation=sim, plane=source_plane, freqs=[freq0], mode_spec=mode_spec)

  modes = ms.solve()

  df = modes.to_dataframe()
  end_time = time.perf_counter()

  # Calculate elapsed time
  elapsed_time = end_time - start_time
  #print("Elapsed time: ", elapsed_time)
  print("DOF: ", len(modes.Ex.y)*len(modes.Ex.z)*2)
  #print(df.head(20))


16:45:51 UTC WARNING: Mode field at frequency index 0, mode index 6 does not    
             decay at the plane boundaries.                                     

16:45:52 UTC WARNING: Mode field at frequency index 0, mode index 7 does not    
             decay at the plane boundaries.                                     

             WARNING: Mode field at frequency index 0, mode index 8 does not    
             decay at the plane boundaries.                                     

             WARNING: Mode field at frequency index 0, mode index 9 does not    
             decay at the plane boundaries.                                     

             WARNING: Mode field at frequency index 0, mode index 10 does not   
             decay at the plane boundaries.                                     

...

DOF:  479440


16:50:02 UTC WARNING: Mode field at frequency index 0, mode index 6 does not    
             decay at the plane boundaries.                                     

             WARNING: Mode field at frequency index 0, mode index 7 does not    
             decay at the plane boundaries.                                     

             WARNING: Mode field at frequency index 0, mode index 8 does not    
             decay at the plane boundaries.                                     

DOF:  746350
